In [ ]:
import pandas as pd
import numpy as np

In [ ]:
pip install contractions

     |████████████████████████████████| 284 kB 41.3 MB/s 
     |████████████████████████████████| 321 kB 76.7 MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85447 sha256=c3e79169fbb5f5f918f503f3f6e2c1febd6f90d2ddc8d18cd010e1dbee5a2298
  Stored in directory: /root/.cache/pip/wheels/25/19/a6/8f363d9939162782bb8439d886469756271abc01f76fbd790f
Successfully built pyahocorasick


In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_6k = pd.read_csv("/content/drive/MyDrive/544_project/data/raw_data/fake_or_real_news.csv")
data_6k = data_6k[['text','label']]
data_6k

,text,label
0,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,It's primary day in New York and front-runners...,REAL
...,...,...
6330,The State Department told the Republican Natio...,REAL
6331,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,FAKE
6332,Anti-Trump Protesters Are Tools of the Oligar...,FAKE
6333,"ADDIS ABABA, Ethiopia —President Obama convene...",REAL


In [ ]:
data_true = pd.read_csv("/content/drive/MyDrive/544_project/data/raw_data/True.csv")
data_true = data_true.drop(['title','subject', 'date'], axis=1)
data_true['label'] = 'REAL'
data_true

,text,label
0,WASHINGTON (Reuters) - The head of a conservat...,REAL
1,WASHINGTON (Reuters) - Transgender people will...,REAL
2,WASHINGTON (Reuters) - The special counsel inv...,REAL
3,WASHINGTON (Reuters) - Trump campaign adviser ...,REAL
4,SEATTLE/WASHINGTON (Reuters) - President Donal...,REAL
...,...,...
21412,BRUSSELS (Reuters) - NATO allies on Tuesday we...,REAL
21413,"LONDON (Reuters) - LexisNexis, a provider of l...",REAL
21414,MINSK (Reuters) - In the shadow of disused Sov...,REAL
21415,MOSCOW (Reuters) - Vatican Secretary of State ...,REAL


In [ ]:
data_false = pd.read_csv("/content/drive/MyDrive/544_project/data/raw_data/Fake.csv")
data_false = data_false.drop(['title','subject', 'date'], axis=1)
data_false['label'] = 'FAKE'
data_false

,text,label
0,Donald Trump just couldn t wish all Americans ...,FAKE
1,House Intelligence Committee Chairman Devin Nu...,FAKE
2,"On Friday, it was revealed that former Milwauk...",FAKE
3,"On Christmas day, Donald Trump announced that ...",FAKE
4,Pope Francis used his annual Christmas Day mes...,FAKE
...,...,...
23476,21st Century Wire says As 21WIRE reported earl...,FAKE
23477,21st Century Wire says It s a familiar theme. ...,FAKE
23478,Patrick Henningsen 21st Century WireRemember ...,FAKE
23479,21st Century Wire says Al Jazeera America will...,FAKE


In [ ]:
data = pd.concat([data_6k, data_false, data_true])
data

,text,label
0,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,It's primary day in New York and front-runners...,REAL
...,...,...
21412,BRUSSELS (Reuters) - NATO allies on Tuesday we...,REAL
21413,"LONDON (Reuters) - LexisNexis, a provider of l...",REAL
21414,MINSK (Reuters) - In the shadow of disused Sov...,REAL
21415,MOSCOW (Reuters) - Vatican Secretary of State ...,REAL


In [ ]:
data = data.sample(frac=1, random_state= 44).reset_index(drop=True)
data

,text,label
0,Donald Trump continues to embarrass himself in...,FAKE
1,NAIROBI (Reuters) - The Kenya Supreme Court on...,REAL
2,"On Tuesday, it was announced that Donald Trump...",FAKE
3,WASHINGTON (Reuters) - Senior Democratic lawma...,REAL
4,The Senate is doing the right thing by not hol...,FAKE
...,...,...
51228,"The fate of the Affordable Care Act, the presi...",REAL
51229,GUATEMALA CITY (Reuters) - Guatemala s Congres...,REAL
51230,Michelle Obama fed her husband s feud with Don...,FAKE
51231,By Martin Winiecki / kosmosjournal.org \nIn th...,FAKE


In [ ]:
train, val, test = \
              np.split(data.sample(frac=1, random_state=42), 
                       [int(.7*len(data)), int(.85*len(data))])

In [ ]:
import contractions
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
def contractionfunction(s):
    words = []
    for word in s.split():
        words.append(contractions.fix(word))
    new_str = ' '.join(words)
    return new_str

# remove the stop words
def rm_stopwords(data):
    stop_words = set(stopwords.words('english'))
    words = [w for w in data.split(' ') if w not in stop_words]
    s = ' '.join(words)
    return s

def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None

# perform lemmatization  
def lemmatize_sentence(sentence):
    lemmatizer = WordNetLemmatizer()
    #tokenize the sentence and find the POS tag for each token
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))  
    #tuple of (token, wordnet_tag)
    wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged)
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            #if there is no available tag, append the token as is
            lemmatized_sentence.append(word)
        else:        
            #else use the tag to lemmatize the token
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    return " ".join(lemmatized_sentence)
    

# Convert the all reviews into the lower case.
# 
# Remove the HTML and URLs from the reviews.
# 
# Remove non-alphabetical characters.
# 
# Remove the extra tab, Line break, spaces, etc. between the words.

def preprocess(data):
  data_after_pro = []
  for r in data:
    # contraction
    r = contractionfunction(r)
    # change all letters into lower case
    r = r.lower()
    # dealing with html
    r = re.sub(r'</?\w+[^>]*>',' ',r)
    # dealing with urls
    r = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+',
               '', r)
    # dealing with non-alphabetical characters
    r = re.sub("[^a-z]+",' ',r)
    # dealing with \n, \b, \r, \t
    r = re.sub(r'\r|\n|\t','',r)
    # dealing with ,/./:/extra spaces....
    r = re.sub(r'[^\w\s]','',r)
    # perform contractions on the reviews
    data_after_pro.append(r) 
  data_after_pro_rmstopwords = []
  for d in data_after_pro:
    data_after_pro_rmstopwords.append(rm_stopwords(d))

  data_after_pro_lemmatize = []
  for d in data_after_pro_rmstopwords:
    data_after_pro_lemmatize.append(lemmatize_sentence(d))

  return data_after_pro_lemmatize

In [ ]:
train_after_pro = preprocess(train['text'])


KeyboardInterrupt: ignored

In [ ]:
dev_after_pro = preprocess(val['text'])
test_after_pro = preprocess(test['text'])

In [ ]:
pd.DataFrame(train_after_pro).to_csv('train_text.txt', header=False, index = False)
pd.DataFrame(test_after_pro).to_csv('test_text.txt', header = False, index = False)
pd.DataFrame(dev_after_pro).to_csv('dev_text.txt', header = False, index = False)

!cp train_text.txt "/content/drive/MyDrive/544_project/data"
!cp test_text.txt "/content/drive/MyDrive/544_project/data"
!cp dev_text.txt "/content/drive/MyDrive/544_project/data"


In [ ]:
train['label']

50850    FAKE
14160    FAKE
44256    REAL
28749    REAL
39459    FAKE
         ... 
7693     FAKE
24949    REAL
50554    REAL
44916    FAKE
16312    FAKE
Name: label, Length: 35863, dtype: object

In [ ]:
train_label = train['label'].apply(lambda x: 0 if x == "FAKE" else 1)
train_label.to_csv('train_label.txt', header=False, index = False)
dev_label = val['label'].apply(lambda x: 0 if x == "FAKE" else 1)
dev_label.to_csv('dev_label.txt', header=False, index = False)
test_label = test['label'].apply(lambda x: 0 if x == "FAKE" else 1)
test_label.to_csv('test_label.txt', header=False, index = False)
!cp train_label.txt "/content/drive/MyDrive/544_project/data"
!cp test_label.txt "/content/drive/MyDrive/544_project/data"
!cp dev_label.txt "/content/drive/MyDrive/544_project/data"